<a href="https://colab.research.google.com/github/fjadidi2001/fake_news_detection/blob/main/DANES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd

# Load the dataset, specifying the encoding as 'latin-1' or 'ISO-8859-1'
dataset_path = "facebook-fact-check.csv"  # Update this path if necessary
data = pd.read_csv(dataset_path, encoding='latin-1') # or encoding='ISO-8859-1'

# Display the first few rows of the dataset
print(data.head())

     account_id       post_id    Category               Page  \
0  1.840000e+14  1.040000e+15  mainstream  ABC News Politics   
1  1.840000e+14  1.040000e+15  mainstream  ABC News Politics   
2  1.840000e+14  1.040000e+15  mainstream  ABC News Politics   
3  1.840000e+14  1.040000e+15  mainstream  ABC News Politics   
4  1.840000e+14  1.040000e+15  mainstream  ABC News Politics   

                                            Post URL Date Published Post Type  \
0  https://www.facebook.com/ABCNewsPolitics/posts...      9/19/2016     video   
1  https://www.facebook.com/ABCNewsPolitics/posts...      9/19/2016      link   
2  https://www.facebook.com/ABCNewsPolitics/posts...      9/19/2016      link   
3  https://www.facebook.com/ABCNewsPolitics/posts...      9/19/2016      link   
4  https://www.facebook.com/ABCNewsPolitics/posts...      9/19/2016     video   

               Rating Debate  share_count  reaction_count  comment_count  \
0  no factual content    NaN          NaN           

In [11]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset, specifying the encoding as 'latin-1' or 'ISO-8859-1'
dataset_path = "facebook-fact-check.csv"  # Update this path if necessary
data = pd.read_csv(dataset_path, encoding='latin-1') # or encoding='ISO-8859-1'

# Display the first few rows of the dataset
print(data.head())

# Display basic information about the dataset
print("\nDataset Information:")
print(data.info())

# Summary statistics
print("\nSummary Statistics:")
print(data.describe())

# Check for missing values
print("\nMissing Values:")
print(data.isnull().sum())

# ---DIAGNOSE---
# Print the available columns to identify the correct target variable name
print("\nAvailable Columns:", data.columns)
# ---DIAGNOSE---

# Visualize the distribution of the target variable (if applicable)
# ---FIX---
# Replace 'label' with the actual column name from your dataset
# Example: If the target variable column is named 'rating', use:
# sns.countplot(x='rating', data=data)
target_variable_column = 'rating'  # Replace with your actual target variable column name

plt.figure(figsize=(8, 6))
sns.countplot(x=target_variable_column, data=data) # Using the updated target_variable_column name
# ---FIX---
plt.title("Distribution of Target Variable")
plt.show()

     account_id       post_id    Category               Page  \
0  1.840000e+14  1.040000e+15  mainstream  ABC News Politics   
1  1.840000e+14  1.040000e+15  mainstream  ABC News Politics   
2  1.840000e+14  1.040000e+15  mainstream  ABC News Politics   
3  1.840000e+14  1.040000e+15  mainstream  ABC News Politics   
4  1.840000e+14  1.040000e+15  mainstream  ABC News Politics   

                                            Post URL Date Published Post Type  \
0  https://www.facebook.com/ABCNewsPolitics/posts...      9/19/2016     video   
1  https://www.facebook.com/ABCNewsPolitics/posts...      9/19/2016      link   
2  https://www.facebook.com/ABCNewsPolitics/posts...      9/19/2016      link   
3  https://www.facebook.com/ABCNewsPolitics/posts...      9/19/2016      link   
4  https://www.facebook.com/ABCNewsPolitics/posts...      9/19/2016     video   

               Rating Debate  share_count  reaction_count  comment_count  \
0  no factual content    NaN          NaN           

ValueError: Could not interpret value `rating` for `x`. An entry with this name does not appear in `data`.

<Figure size 800x600 with 0 Axes>